In [1]:
!pip install pandas

In [3]:
import pandas as pd
file = pd.read_csv('submission_metadata.csv')

In [4]:

file = file.drop_duplicates(subset=['Submission ID'])
file.reset_index(drop=True, inplace=True)
file

,First Name,Last Name,Student ID,Email,Status,Submission ID,Total Score,Max Points,Question 1 Score,Question 1 Weight,...,Question 14 Score,Question 14 Weight,Question 14 Graded?,Question 14 Response,Question 14 Submitted At,Question 15 Score,Question 15 Weight,Question 15 Graded?,Question 15 Response,Question 15 Submitted At
0,Zhengrui,Wang,68798785,wzray@seas.upenn.edu,Ungraded,283470915.0,10.0,10.0,0.0,0.0,...,5.0,5.0,True,{},2024-10-29 17:49:30 -0700,5.0,5.0,True,{},2024-10-29 17:49:30 -0700
1,Rachel,Doman,31356120,rdoman@wharton.upenn.edu,Ungraded,284077241.0,10.0,10.0,0.0,0.0,...,5.0,5.0,True,{},2024-10-31 07:36:55 -0700,5.0,5.0,True,{},2024-10-31 07:36:55 -0700
2,Etienne,Lee,40487396,etienne9@seas.upenn.edu,Ungraded,284364504.0,10.0,10.0,0.0,0.0,...,5.0,5.0,True,{},2024-10-31 20:07:34 -0700,5.0,5.0,True,{},2024-10-31 20:07:34 -0700
3,Zhiyu,Liu,18148698,liuzhiyu@seas.upenn.edu,Ungraded,283602865.0,10.0,10.0,0.0,0.0,...,5.0,5.0,True,{},2024-10-30 01:02:02 -0700,5.0,5.0,True,{},2024-10-30 01:02:02 -0700
4,Vihaan,Jain,45603181,vihaanj@wharton.upenn.edu,Ungraded,284188088.0,10.0,10.0,0.0,0.0,...,5.0,5.0,True,{},2024-10-31 12:24:23 -0700,5.0,5.0,True,{},2024-10-31 12:24:23 -0700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Jadyn,Elliott,29645516,jadyne@seas.upenn.edu,Ungraded,284057240.0,10.0,10.0,0.0,0.0,...,5.0,5.0,True,{},2024-10-31 05:44:47 -0700,5.0,5.0,True,{},2024-10-31 05:44:47 -0700
113,Aidan,Crowley,79535511,aidan.crowley@pennmedicine.upenn.edu,Ungraded,283773402.0,10.0,10.0,0.0,0.0,...,5.0,5.0,True,{},2024-10-30 12:51:50 -0700,5.0,5.0,True,{},2024-10-30 12:51:50 -0700
114,Forest,Ho-Chen,39221160,foresth@seas.upenn.edu,Ungraded,283825868.0,10.0,10.0,0.0,0.0,...,5.0,5.0,True,{},2024-10-30 14:45:06 -0700,5.0,5.0,True,{},2024-10-30 14:45:06 -0700
115,Swarith,Reddy,82506347,swarithr@wharton.upenn.edu,Ungraded,283421044.0,10.0,10.0,0.0,0.0,...,5.0,5.0,True,{},2024-10-29 15:38:33 -0700,5.0,5.0,True,{},2024-10-29 15:38:33 -0700


In [6]:
import pandas as pd
import random
import numpy as np

# Load your data (replace 'path_to_file.csv' with the actual file path)
# df = pd.read_csv('path_to_file.csv')  # Uncomment if loading from CSV

# Sample TA list
tas = [
    "Emily Liu", "Peter Akioyamen", "Kyle Liao", "Michael Lu", "Praj Chirathivat",
    "Vickie Liu", "Jessica Forsstrom", "Oscar Eichmann", "Aeshon Balasubramanian",
    "William Qi", "Term Taepaisitphongse", "Steven Su", "Henry Sims", "Hassan Rizwan",
    "Alan Wu", "Ming Cao", "Bekzat Amirbay", "Jon Wu"
]

# Initialize the dictionary for TA groups
ta_groups = {ta: [] for ta in tas}

# Function to assign a student to any available TA if preferences are "No Preference" or NaN
def assign_random_ta(student_index):
    available_tas = [ta for ta, students in ta_groups.items() if len(students) < 7]
    if available_tas:
        random_ta = random.choice(available_tas)
        ta_groups[random_ta].append(student_index)

# Step 1: Initial assignment based on preferences
for index, row in file.iterrows():
    first_choice = row['Question 11 Response']
    second_choice = row['Question 12 Response']
    third_choice = row['Question 13 Response']
    
    if pd.notna(first_choice) and first_choice != "No Preference" and len(ta_groups[first_choice]) < 6:
        ta_groups[first_choice].append(index)
    elif pd.notna(second_choice) and second_choice != "No Preference" and len(ta_groups[second_choice]) < 6:
        ta_groups[second_choice].append(index)
    elif pd.notna(third_choice) and third_choice != "No Preference" and len(ta_groups[third_choice]) < 6:
        ta_groups[third_choice].append(index)
    else:
        assign_random_ta(index)

# Step 2: Balance groups by redistributing from max to min until all have at least 5 students
while True:
    # Sort TAs by the number of assigned students
    sorted_tas = sorted(ta_groups.items(), key=lambda x: len(x[1]))
    min_ta, min_students = sorted_tas[0]
    max_ta, max_students = sorted_tas[-1]
    
    # Stop if the minimum group has at least 5 students
    if len(min_students) >= 6:
        break

    # Pop a student from the max TA and add to the min TA
    student_to_move = ta_groups[max_ta].pop()
    ta_groups[min_ta].append(student_to_move)

# Step 3: Prepare data for CSV export with selected columns
selected_columns = [
    'Question 2 Response', 'Question 3 Response', 'Question 4 Response', 
    'Question 5 Response', 'Question 6 Response', 'Question 7 Response', 
    'Question 8 Response', 'Question 9 Response', 'Question 10 Response'
]
export_data = []
for ta, students in ta_groups.items():
    for student_index in students:
        row = file.loc[student_index, selected_columns].to_dict()  # Get only selected columns for the student
        row['TA'] = ta  # Add the TA assignment to the row
        export_data.append(row)

# Convert to DataFrame for easy export
export_df = pd.DataFrame(export_data)

# Export to CSV with assigned students
export_df.to_csv('ta_assignment_selected_columns.csv', index=False)

# Step 4: Print and export summary of the number of groups assigned to each TA
summary_data = [{'TA': ta, 'Number of Students': len(students)} for ta, students in ta_groups.items()]
summary_df = pd.DataFrame(summary_data)

# Print summary
print("TA Assignment Summary:")
print(summary_df)

# Export summary to CSV
summary_df.to_csv('ta_assignment_summary.csv', index=False)
print("Assignment summary has been exported to 'ta_assignment_summary.csv'")


TA Assignment Summary:
                        TA  Number of Students
0                Emily Liu                   7
1          Peter Akioyamen                   6
2                Kyle Liao                   7
3               Michael Lu                   7
4         Praj Chirathivat                   7
5               Vickie Liu                   7
6        Jessica Forsstrom                   7
7           Oscar Eichmann                   6
8   Aeshon Balasubramanian                   5
9               William Qi                   6
10   Term Taepaisitphongse                   6
11               Steven Su                   7
12              Henry Sims                   6
13           Hassan Rizwan                   7
14                 Alan Wu                   7
15                Ming Cao                   7
16          Bekzat Amirbay                   6
17                  Jon Wu                   6
Assignment summary has been exported to 'ta_assignment_summary.csv'
